# Pytorch NER Bilstm CRF Tutorial

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchtext
from torchtext.data import Field, BucketIterator
from torchtext.datasets import UDPOS
from torchcrf import CRF

from tqdm import tqdm
import spacy
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
spacy_en = spacy.load("en_core_web_sm")

In [4]:
def tokenizer_en(text):
    return [token.text for token in spacy_en.tokenizer(text.lower())]

In [5]:
TEXT = Field(
    sequential=True, 
    tokenize=tokenizer_en,
#     init_token="<sos>",
    eos_token="<eos>",
#     batch_first=True,
    lower=True
)
LABELS = Field(
    sequential=True, 
#     init_token="<sos>",
    eos_token="<eos>",
#     batch_first=True,
    is_target=True
)
# PTB_LABELS = Field(
#     sequential=True, 
#     init_token="<sos>",
#     eos_token="<eos>",
# #     batch_first=True,
#     is_target=True
# )

In [6]:
training_data, validation_data, test_data = UDPOS.splits(fields=(("text", TEXT), ("labels", LABELS), (None, None)))
len(training_data), len(validation_data), len(test_data)

(12543, 2002, 2077)

In [7]:
example = vars(training_data.examples[0])
# example
print("sentence : {}".format(" ".join(example["text"])))
print("pos : {}".format(" ".join(example["labels"])))

sentence : al - zaman : american forces killed shaikh abdullah al - ani , the preacher at the mosque in the town of qaim , near the syrian border .
pos : PROPN PUNCT PROPN PUNCT ADJ NOUN VERB PROPN PROPN PROPN PUNCT PROPN PUNCT DET NOUN ADP DET NOUN ADP DET NOUN ADP PROPN PUNCT ADP DET ADJ NOUN PUNCT


In [8]:
TEXT.build_vocab(training_data, vectors="glove.6B.100d")
LABELS.build_vocab(training_data)
# PTB_LABELS.build_vocab(training_data)

len(TEXT.vocab), len(LABELS.vocab)

(16656, 20)

In [9]:
training_iterator, val_iterator, test_iterator = BucketIterator.splits(
    (training_data, validation_data, test_data),
    batch_sizes=(128, 32, 32), device=device
)
len(training_iterator), len(test_iterator), len(val_iterator)

(98, 65, 63)

In [10]:
for batch in training_iterator:
    break

In [11]:
batch.text.shape

torch.Size([49, 128])

In [12]:
# batch.labels

In [13]:
LABELS.vocab.itos[2], LABELS.vocab.itos[3]

('<eos>', 'NOUN')

In [14]:
# vocab_size = len(TEXT.vocab)
# embedding_dim = 10
# hidden_dim = 5
# tagset_size = len(LABELS.vocab)

# embedding = nn.Embedding(vocab_size, embedding_dim)
# lstm = nn.LSTM(embedding_dim, hidden_dim)
# linear_layer = nn.Linear(hidden_dim, tagset_size)
# crf_layer = CRF(tagset_size)

# print("Input X shape is {}".format(batch.text.shape))

# embedded = embedding(batch.text)
# print("Shape of embedded is {}".format(embedded.shape))

# lstm_out , (_, _) = lstm(embedded)
# print("Shape of lstm out is {}".format(lstm_out.shape))

# emmissions = linear_layer(lstm_out)
# print("Emmission output shape is {}".format(emmissions.shape))

# loss = crf_layer(emmissions, batch.labels)
# print(loss.item())

# crf_out = crf_layer.decode(emmissions)
# crf_out

# torch.tensor(crf_out, dtype=torch.long, device=device).permute(1, 0).shape

# LABELS.vocab.itos[15], LABELS.vocab.itos[6], LABELS.vocab.itos[14], LABELS.vocab.itos[0]

In [15]:
class BiLstm_Crf(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, bidirectional):
        super(BiLstm_Crf, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.hidden_dim, num_layers=self.num_layers, bidirectional=self.bidirectional,
                           dropout=0.5)
        
        self.dropout_layer = nn.Dropout(0.5)
        self.linear = nn.Linear(self.hidden_dim, self.output_dim)
        
        self.crf_layer = CRF(self.output_dim)
        self.inference = False
        
    def forward(self, inp, labels):
        # inp = [seq_len, batch_size]
        # labels = [seq_len, batch_size]
             
        embedded = self.dropout_layer(self.embedding(inp))
        # embedded = [seq_len, batch_size, embedding_dim]
        
        outputs, (hidden, cell) = self.lstm(embedded)
        # outputs = [seq_len, batch_size, 1 * hidden_size]
        
        out = self.linear(outputs)
        # out = [seq_len, batch_size, output_dim]
        
        if self.inference is False:
            loss = self.crf_layer(out, labels) * torch.tensor(-1, device=device)
            return loss 
        else:
            loss = self.crf_layer(out, labels) * torch.tensor(-1, device=device)
            out = self.crf_layer.decode(out)
            out = torch.tensor(out, dtype=torch.long, device=device).permute(1, 0)
            # out = [seq_len, batch_size]
            return out, loss

In [16]:
def train(model, iterator, optimizer, device=None):
    model.train()
    model.inference = False
    
    epoch_loss = 0.0 
    
    for batch in tqdm(iterator):
        inp = batch.text
        target = batch.labels
        
        optimizer.zero_grad()
        
        loss = model(inp, target)
        # crf loss
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, device=None):
    model.eval()
    
    epoch_loss = 0.0
    predictions = []
    true_labels = []
    model.inference = True
    for batch in tqdm(iterator):
        inp = batch.text
        target = batch.labels
                
        out, loss = model(inp, target)
        # out = [seq_len, batch_size]
        # crf loss
        
        predictions.extend(out.contiguous().view(-1).cpu().tolist())
        true_labels.extend(target.contiguous().view(-1).cpu().tolist())
                
        epoch_loss += loss.item()
    
    f1 = f1_score(true_labels, predictions, average="macro")
        
    return epoch_loss / len(iterator), f1

def number_of_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [17]:
model = BiLstm_Crf(
    vocab_size=len(TEXT.vocab), 
    embedding_dim=100, hidden_dim=512, 
    output_dim=len(LABELS.vocab), 
    num_layers=2, bidirectional=False
)

In [18]:
model.embedding.weight.data.copy_(TEXT.vocab.vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2634,  0.0742, -0.1081,  ..., -0.2977, -0.5655,  0.5218],
        [ 0.4244,  0.6004, -0.1528,  ...,  0.2536, -0.4969,  0.8964]])

In [19]:
number_of_parameters(model)

5035020

In [20]:
# criterion = nn.CrossEntropyLoss(ignore_index=LABELS.vocab.stoi["<pad>"])
optimizer = optim.Adam(model.parameters())

In [21]:
model = model.to(device)
# criterion = criterion.to(device)

In [22]:
N_EPOCHS = 50
model.inference = False
VAL_LOSS = 1e10
for epoch in range(N_EPOCHS):
    train_loss = train(model, training_iterator, optimizer)
    val_loss, val_f1 = evaluate(model, val_iterator)
    
    if VAL_LOSS > val_loss:
        VAL_LOSS = val_loss
        torch.save(model.state_dict(), 'bilstm-ner-crf-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val. Loss: {val_loss:.3f}')
    print(f'Val. F1 Score is : {val_f1:.2f}')
    torch.cuda.empty_cache() 

  0%|          | 0/98 [00:00<?, ?it/s]

Epoch: 01 | Train Loss: 6329.186 | Val. Loss: 624.330
Val. F1 Score is : 0.31


  1%|          | 1/98 [00:00<00:15,  6.16it/s]

Epoch: 02 | Train Loss: 2378.740 | Val. Loss: 288.909
Val. F1 Score is : 0.62


  1%|          | 1/98 [00:00<00:13,  6.96it/s]

...

Epoch: 48 | Train Loss: 167.204 | Val. Loss: 115.348
Val. F1 Score is : 0.87


  1%|          | 1/98 [00:00<00:13,  7.37it/s]

Epoch: 49 | Train Loss: 162.263 | Val. Loss: 117.863
Val. F1 Score is : 0.87


100%|██████████| 63/63 [00:01<00:00, 50.88it/s] 

Epoch: 50 | Train Loss: 157.926 | Val. Loss: 121.868
Val. F1 Score is : 0.87


In [23]:
model.load_state_dict(torch.load('bilstm-ner-crf-model.pt'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [24]:
for i, batch in enumerate(val_iterator):
    model.eval()
    model.inference = True
    inp = batch.text[:, 1].unsqueeze(1)
    targ = batch.labels[:, 1]
#     print(inp.tolist())
    outputs, loss = model(inp, targ.unsqueeze(1))
    # outputs = [seq_len, batch_size]
#     print(outputs.view(-1).tolist())
    print("text = {}".format(" ".join([TEXT.vocab.itos[w] for w in list(inp.view(-1).tolist())])))
    sentence = " ".join([LABELS.vocab.itos[w] for w in list(outputs.view(-1).tolist())])
    print("Output = {}".format(sentence))
    print("Truth = {}".format(" ".join([LABELS.vocab.itos[w] for w in list(targ.view(-1).tolist())])))
    print()
    if i == 100:
        break

text = <unk> <eos>
Output = X <eos>
Truth = PROPN <eos>

text = julie <eos>
Output = PROPN <eos>
Truth = PROPN <eos>

text = professional <eos>
Output = ADJ <eos>
Truth = ADJ <eos>

text = thanks . <eos>
Output = NOUN PUNCT <eos>
Truth = NOUN PUNCT <eos>

text = <unk> <unk> <eos>
Output = PROPN PROPN <eos>
Truth = PROPN PROPN <eos>

text = bill <unk> <eos>
Output = PROPN PROPN <eos>
Truth = PROPN PROPN <eos>

text = <unk> ! <eos>
Output = NOUN PUNCT <eos>
Truth = ADJ PUNCT <eos>

text = <unk> <unk> am <eos>
Output = PROPN PROPN AUX <eos>
Truth = NUM NUM NOUN <eos>

text = absolutely free . <eos>
Output = ADV ADJ PUNCT <eos>
Truth = ADV ADJ PUNCT <eos>

text = <unk> - <unk> <eos>
Output = NOUN PUNCT PROPN <eos>
Truth = NUM PUNCT NOUN <eos>

text = thanks from bill <eos>
Output = NOUN ADP PROPN <eos>
Truth = NOUN ADP PROPN <eos>

text = <unk> was awesome . <eos>
Output = NOUN AUX ADJ PUNCT <eos>
Truth = PROPN AUX ADJ PUNCT <eos>

text = mix it all up <eos>
Output = NOUN PRON DET ADP <eos

In [37]:
for i, batch in enumerate(val_iterator):
    break

In [75]:
class AverageMeter(object):
    """
    Keeps track of most recent, average, sum, and count of a metric.
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [38]:
model.eval()
model.inference = True

In [39]:
outputs = model(batch.text[:, 1].unsqueeze(1), None)
outputs.shape

TypeError: ones_like() received an invalid combination of arguments - got (NoneType, dtype=torch.dtype), but expected one of:
 * (Tensor input, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (Tensor input, bool requires_grad)
      didn't match because some of the keywords were incorrect: dtype


In [78]:
f1s = AverageMeter()

In [93]:
batch.labels[:, 1].to("cpu").tolist()

[9, 2]

In [95]:
outputs.view(-1).to("cpu").tolist()

[3, 2]

In [96]:
f1 = f1_score(batch.labels[:, 1].to("cpu").tolist(), outputs.view(-1).to("cpu").tolist(), average=None)

/data/users-workspace/kuldeep.singh/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/data/users-workspace/kuldeep.singh/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [97]:
f1

array([1., 0., 0.])

In [81]:
f1s.update(f1)

In [83]:
f1s.val

0.3333333333333333